<a href="https://colab.research.google.com/github/MedEssalemy/EtsyAnalysis/blob/main/ColumnsMatching_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

*Install the required libraries before using the notebook (only on the first use).*


In [ ]:
pip install pandas openpyxl sentence-transformers fuzzywuzzy

# **Process Data**

*Import the raw data file in the standard format, then launch the process data code*

In [20]:
# @title Import Working Data
from google.colab import files
import os
import shutil

def upload_files(default_message="Please upload your files to proceed:"):
    print(default_message)
    uploaded = files.upload()

    if not uploaded:
        print("\nUpload canceled. No files were uploaded.")
        return None

    print("\nFiles uploaded successfully!")
    for filename in uploaded.keys():
        print(f" - {filename}")
    return uploaded

def save_uploaded_file(uploaded_files):
    if uploaded_files is None:
        print("No files to save.")
        return

    # Define paths
    target_folder = '/content/sample_data'
    target_filename = 'raw_data.xlsx'
    target_path = os.path.join(target_folder, target_filename)

    # Get the first uploaded file
    uploaded_filename = next(iter(uploaded_files.keys()))

    # Remove the file from root directory if it exists
    if os.path.exists(uploaded_filename):
        os.remove(uploaded_filename)

    # Save directly to sample_data folder
    with open(target_path, 'wb') as f:
        f.write(uploaded_files[uploaded_filename])

    print(f"File saved as '{target_filename}' in the 'sample_data' folder.")

# Call the upload function
uploaded_files = upload_files()

# Save the uploaded file to the sample_data folder
save_uploaded_file(uploaded_files)

Please upload your files to proceed:


Saving data - Copy.xlsx to data - Copy.xlsx

Files uploaded successfully!
 - data - Copy.xlsx
File saved as 'raw_data.xlsx' in the 'sample_data' folder.


In [21]:
# @title Process Data

import pandas as pd
import re
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer, util
import os
from google.colab import files  # Import for downloading files
from datetime import datetime

# Load the semantic model
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

def preprocess(text):
    text = str(text) if text is not None else ""
    text = re.sub(r'((?<=[a-z0-9])[A-Z]|(?<=[A-Z])[A-Z](?=[a-z]))', r' \1', text)
    text = text.replace('_', ' ').replace('-', ' ')
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return ' '.join(text.lower().split())

def calculate_similarity(str1, str2):
    if str1 == str2:
        return 100
    return fuzz.ratio(str1, str2)

def calculate_semantic_similarity_batch(sentences1, sentences2):
    embeddings1 = semantic_model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = semantic_model.encode(sentences2, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return scores.cpu().numpy() * 100

def normalize_column_mapping_generator(excel_file, threshold=50):
    master_df = pd.read_excel(excel_file, sheet_name='Master')
    search_df = pd.read_excel(excel_file, sheet_name='Search')

    master_data = [
        {
            'platform': row['Platform'],
            'orig_table': row['Table Name'],
            'orig_column': row['Column Name'],
            'clean_table': preprocess(row['Table Name']),
            'clean_column': preprocess(row['Column Name'])
        }
        for _, row in master_df.iterrows()
    ]

    search_data = [
        {
            'orig_table': row['Table Name'],
            'orig_column': row['Column Name'],
            'source_sheet': row['Source Sheet'],
            'clean_table': preprocess(row['Table Name']),
            'clean_column': preprocess(row['Column Name'])
        }
        for _, row in search_df.iterrows()
    ]

    # Precompute semantic embeddings
    master_table_embeddings = semantic_model.encode(
        [entry['clean_table'] for entry in master_data], convert_to_tensor=True
    )
    master_column_embeddings = semantic_model.encode(
        [entry['clean_column'] for entry in master_data], convert_to_tensor=True
    )
    search_table_embeddings = semantic_model.encode(
        [entry['clean_table'] for entry in search_data], convert_to_tensor=True
    )
    search_column_embeddings = semantic_model.encode(
        [entry['clean_column'] for entry in search_data], convert_to_tensor=True
    )

    for master_idx, master_entry in enumerate(master_data):
        for search_idx, search_entry in enumerate(search_data):
            table_similarity = calculate_similarity(
                master_entry['clean_table'], search_entry['clean_table']
            )
            column_similarity = calculate_similarity(
                master_entry['clean_column'], search_entry['clean_column']
            )

            if table_similarity == 100:
                semantic_table_similarity = 100
            else:
                semantic_table_similarity = util.pytorch_cos_sim(
                    master_table_embeddings[master_idx], search_table_embeddings[search_idx]
                ).item() * 100

            if column_similarity == 100:
                semantic_column_similarity = 100
            else:
                semantic_column_similarity = util.pytorch_cos_sim(
                    master_column_embeddings[master_idx], search_column_embeddings[search_idx]
                ).item() * 100

            if column_similarity >= threshold or semantic_column_similarity >= threshold:
                yield {
                    'Platform': master_entry['platform'],
                    'Master Table': master_entry['orig_table'],
                    'Search Table': search_entry['orig_table'],
                    'Search Sheet': search_entry['source_sheet'],
                    'Master Column': master_entry['orig_column'],
                    'Search Column': search_entry['orig_column'],
                    'Table Similarity (%)': round(table_similarity, 2),
                    'Column Similarity (%)': round(column_similarity, 2),
                    'Semantic Table Similarity (%)': round(semantic_table_similarity, 2),
                    'Semantic Column Similarity (%)': round(semantic_column_similarity, 2)
                }

def export_incremental_column_mappings(excel_file, output_file, threshold=50, increment=10000):
    if os.path.exists(output_file):
        previous_df = pd.read_excel(output_file)
        previous_mappings = previous_df.to_dict('records')
    else:
        previous_mappings = []

    mapping_generator = normalize_column_mapping_generator(excel_file, threshold)
    current_mappings = {frozenset(mapping.items()): mapping for mapping in previous_mappings}
    new_mappings = []

    for mapping in mapping_generator:
        mapping_hash = frozenset(mapping.items())
        if mapping_hash not in current_mappings:
            new_mappings.append(mapping)
            current_mappings[mapping_hash] = mapping

        if len(new_mappings) >= increment:
            df = pd.DataFrame(list(current_mappings.values()))
            df.to_excel(output_file, index=False)
            print(f"Exported {len(current_mappings)} cumulative mappings")
            new_mappings = []

    if new_mappings:
        df = pd.DataFrame(list(current_mappings.values()))
        df.to_excel(output_file, index=False)
        print(f"Final export of {len(current_mappings)} mappings")

    # Download the file after export
    print("\nDownloading the results to your local machine...")
    files.download(output_file)

# Main
if __name__ == "__main__":
    excel_file_path = 'sample_data/raw_data.xlsx'
    output_file_path = f'sample_data/processed_data_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
    export_incremental_column_mappings(excel_file_path, output_file_path, threshold=50)


Exported 10000 cumulative mappings
Final export of 15081 mappings



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>